In [4]:
import json
import sqlite3
import pandas as pd
import config
import numpy as np
import requests
import sys

from sqlalchemy import create_engine
engine = create_engine('postgresql://'+config.db['user']+':'+config.db['password']+config.db['url'])

sSource = "file"
if sSource=="file":
    with open('./data/NVDRstock-trading31-01-23.json','r') as f:
        data = json.loads(f.read())
elif sSource=="api":
    #r = requests.get('https://www.set.or.th/api/set/nvdr-trade/stock-trading?sortBy=symbol')
    url = 'https://www.set.or.th/api/set/nvdr-trade/stock-trading?sortBy=symbol'
    headers = {
        'Content-Type': 'application/json',
        'Host': 'www.set.or.th',
        'Origin': 'https:///www.set.or.th',
        'Referer' : 'https://www.set.or.th/th/market/statistics/nvdr/trading-by-stock',
        'Cookie': 'id=KqKgOXZeJxapT8k8AL1rOg%3D%3D00000000;'
        }
    r = requests.get(url,headers=headers)
    print("response: " + r.text)
    if r.status_code == 200:
        data = r.json()
    else:
        print("Error from server: " + str(r.content))
        exit



response: 
Error from server: b''


In [ ]:
#df = pd.json_normalize(r.json(), record_path =['nvdrTradings'])

cdate = pd.DataFrame(data).iloc[0].astype(str).str[:10]['date']
#df.info()
print(cdate)
df = pd.json_normalize(data, record_path =['nvdrTradings'])
df.drop(['buyVolume', 'sellVolume','netVolume', 'totalVolume','underlyingVolume', 'percentVolume'], axis=1, inplace=True)


#print(df)
df.rename(columns = {'date': 'trddate', 'buyValue': 'buy', 'sellValue': 'sell', 'netValue': 'net', 'totalValue': 'total', 'percentValue': 'percent', 'underlyingValue':'trdvalue'}, inplace = True)
df['ft']=df['total']/df['trdvalue']*100
df['ft'].round(2)
#df.info()
print(df)

In [ ]:
sql = "select * from snvdr where trddate = '"+ cdate + "'"
print(sql)
df = df.loc[~((df['buy'] == '-') | (df['sell'] == '-') | (df['net'] == '-'))]
print(df)
rst = pd.read_sql(sql, con=engine)
print(len(rst.axes[0]))
if len(rst.axes[0]) == 0:df.to_sql(name='snvdr', con=engine, if_exists='append')
cur = engine.connect()
cur.execute("DELETE FROM snvdrsum")
cur.execute("INSERT into snvdrsum(Symbol, sumn, counts) select symbol,round(sum(net)) as sumn,count(symbol)counts from snvdr group by symbol")

# sqlAvg = "UPDATE snvdr SET  \
# ft=(SELECT round(abs(n1.net) / (s.value + 1::numeric) / 10::numeric, 1) as tf  \
# FROM snvdr n1 inner join sprice s on n1.symbol = s.series and n1.TrdDate=s.trddate where n1.Symbol = snvdr.symbol AND n1.TrdDate = (SELECT DISTINCT TrdDate FROM ssfOI ORDER by TrdDate DESC  LIMIT 1 OFFSET 0)), \
# calvol=(SELECT round((n1.net) / ((s.open+s.close)/2::numeric), 1) as calvol   \
# FROM snvdr n1 inner join sprice s on n1.symbol = s.series and n1.TrdDate=s.trddate where n1.Symbol = snvdr.symbol AND n1.TrdDate = (SELECT DISTINCT TrdDate FROM ssfOI ORDER by TrdDate DESC  LIMIT 1 OFFSET 0))  \
# where snvdr.TrdDate = (SELECT DISTINCT TrdDate FROM ssfOI ORDER by TrdDate DESC  LIMIT 1 OFFSET 0)"
# print(sqlAvg)
#cur.execute(sqlAvg)

In [ ]:
df.info()

In [ ]:
import pandas as pd
#import datetime
import sqlite3
# Create your connection.
cnx = sqlite3.connect('EquityTrend.db')
df=pd.read_html('https://www.set.or.th/set/nvdrbystock.do?type=value&sort=total&language=en&country=US')
df=df[0]
#df
df.columns = df.columns.droplevel()
df.rename(columns = {'%*':'Percent'}, inplace = True)

In [ ]:
#df = df.drop(df.columns[[1,2]],axis='columns')
df

In [ ]:
df['TrdDate'] = pd.to_datetime('today').date()
df['TrdDate'] = df['TrdDate'].apply(pd.DateOffset(-1)).apply(lambda x : x.date())
#df['TrdDate']= df['TrdDate']
df

In [ ]:
sql = "select * from sNVDR where TrdDate = '"+ df.iloc[0,6].strftime('%Y-%m-%d') + "'"
print(sql)
rst = pd.read_sql(sql, cnx)
print(len(rst.axes[0]))
#if len(rst.axes[0]) == 0:df.to_sql(name='sNVDR', con=cnx, if_exists='append')

In [ ]:
df = df.iloc[0:50]

In [ ]:
df.info()
df

In [ ]:
df.loc[df['Symbol'] == 'TFI']